# FSM

## Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
raw_df = pd.read_csv('../data/archive/1429_1.csv')
    
df = raw_df[['asins', 'reviews.rating', 'reviews.doRecommend', 'reviews.title', 'reviews.text']]
    
df.dropna(inplace=True)

/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
df.sample(3)

,asins,reviews.rating,reviews.doRecommend,reviews.title,reviews.text
7377,B018Y229OU,4.0,True,Great product,Great product compact and easy to get around i...
139,B01AHB9CN2,5.0,True,Best bang for your buck!,Amazon Fire HD8 is a great product for the pri...
24474,"B00L9EPT8O,B01E6AO69U",5.0,True,Tons of fun,"I love asking Alexa to turn on my music, alarm..."


## Cleaning

In [5]:
sw = stopwords.words('english')

In [6]:
#from last NLP project, written with Amanda Potter 
def doc_preparer(doc, stop_words=sw):
    '''
    
    :param doc: a document from the corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in stop_words]
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

In [7]:
#from last NLP project, written with Amanda Potter 
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
  

def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [8]:
X = df['reviews.text']
y = df['reviews.doRecommend']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12, test_size=.15)

In [9]:
#b = "why are you doing this to me git, why"

In [10]:
X_train_dp = [doc_preparer(doc, sw) for doc in X_train]

In [12]:
print(type(X_train_dp))
X_train_dp[0]

<class 'list'>


'told fire stick tv friend go best buy store buy one store order one hand two day enjoy use everyday'

I'm seeing words like 'go' which will likely not be of any signifigance, I'll need to do an analysis of the word counts later on before making a 'better model'

In [13]:
X_t, X_val, y_t, y_val = train_test_split(X_train_dp, y_train, random_state=12, test_size=.20)

In [14]:
cv = CountVectorizer()

X_t_cv = cv.fit_transform(X_t)
X_t_cv = pd.DataFrame.sparse.from_spmatrix(X_t_cv)
X_t_cv.columns = sorted(cv.vocabulary_)
X_t_cv.set_index(y_t.index, inplace=True) #keeping it orderly 

X_t_cv #this might look more orderly than last time 

,aa,aamazon,aand,abandon,abc,abcs,abd,ability,abilty,abke,...,zigbee,zip,zipper,zippy,zombie,zone,zoo,zoom,zooming,zwave
15806,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8314,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


It worked! Now time to put it through the ol' naive bayes 

In [15]:
val_0 = None
val_1 = None